In [1]:
import math
import numpy as np
import pygame
from tqdm import tqdm
from map import Map
from heapq import heappush, heappop
import time

pygame 2.1.2 (SDL 2.0.18, Python 3.9.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#Variables
arr_name = '2024n1'
robot_dimension = 26
image_name = 'images/n2024.png'

In [3]:
#code to add circles to array
load_path = 'arrays/'+arr_name+'.npy'
circle_radius = 2
print(circle_radius)

#Euclidean Distance
def dist(x1, y1, x2, y2):
    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

#Makes a circle
def make_circle(arr, cx, cy, r):
    for x in range(cx - r, cx + r):
        for y in range(cy - r, cy + r):
            if dist(cx, cy, x, y) <= r:
                if x < 59 and y < 29:
                    arr[x][y] = 1

arr = np.load(load_path)
#print(arr.shape)
new_arr = np.zeros((59,29))
num = 0
for i in tqdm(range(59)):
    for c in range(29):
        if arr[i][c] == 1:
            make_circle(new_arr, i, c, circle_radius)
            num = num + 1
arr = new_arr
print(num)



2


100%|██████████| 59/59 [00:00<00:00, 19401.33it/s]

102


In [4]:
#Node class with parent, g, f, and h
class Node():
    def __init__(self, parent=None, pos=None):
        self.parent = parent
        self.position = pos

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position
    def __lt__(self, other):
        return self.f < other.f

In [5]:
#All of my heuristics

#Euclidean Distance
def euclideanDistance (child, end):
    dx = abs(child.position[0] - end[0])
    dy = abs(child.position[1]- end[1])
    return int(math.sqrt(dx**2 + dy**2)*10)

#Diagonal Distance, Chebyshev
def diagonalDistanceChebyshev(child, end):
    dx = abs(child.position[0] - end[0])
    dy = abs(child.position[1]-end[1])
    #d = 1, d2 = 1 (d,d2,d)
    return int((1 * (dx + dy) + (1 - 2 * 1) * min(dx, dy))*10)

#Diagonal Distance, Octile(Difference in the d2, its sqrt 2)
def diagonalDistanceOctile(child, end):
    dx = abs(child.position[0] - end[0])
    dy = abs(child.position[1]-end[1])
    #d = 1, d2 = 1 (d,d2,d)
    return int((1 * (dx + dy) + (math.sqrt(2) - 2 * 1) * min(dx, dy))*10)

In [6]:
#A* function(not used)
def astar(arr, start, end):
    openlist = []
    closedlist = []
    
    start_node = Node(None, start)
    start_node.g = start_node.h = start_node.f = 0
    end_node = Node(None, end)
    end_node.g = end_node.h = end_node.f = 0
    
    heappush(openlist, start_node)
    
    while len(openlist) > 0:
        breakbool, openlist, closedlist, final_node = astarloop(arr, openlist, closedlist, end)
    
        if breakbool == True:
            path = []
            path.insert(0,final_node.position)
            node = final_node.parent
            while node != start:
                path.insert(0, node.position)
                node = node.parent
            return path
      
#A* loop function, takes open and closed lists and returns them
#One iteration of the A* algorithm  
def astarloop(arr, openlist, closedlist, end):
    current_node = heappop(openlist)
    closedlist.append(current_node)
    #break if found final path
    if current_node.position == end:
        return True, openlist, closedlist, current_node
    #iterate through all possible neighbors
    for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]:
        skip = False
        pos = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])
        new_node = Node(None, pos)
        #checking to see if it's in closed list
        for check in closedlist:
            if new_node == check:
                #print("skipping")
                skip = True
        if not skip:
            if arr[(new_node.position[0], new_node.position[1])] == 1:
                #print("skipping")
                skip = True
            if not skip:
                new_node.parent = current_node
                if new_position == (0, -1) or new_position == (1, 0) or new_position == (0, 1) or new_position == (-1, 0):
                    new_node.g = current_node.g + 10
                else:
                    new_node.g = current_node.g + 14
                new_node.h = euclideanDistance(new_node, end)
                new_node.f = new_node.g + new_node.h
                for check in openlist:
                    if new_node == check:
                        if new_node.g < check.g:
                            check.g = new_node.g
                            check.f = new_node.f
                            check.parent = current_node
                        #print('skipping')
                        skip = True
                if not skip:
                    #print("pushing")
                    heappush(openlist, new_node)
    return False, openlist, closedlist, current_node

In [ ]:
# checkPoint = starting point of path
# currentPoint = next point in path
# while (currentPoint->next != NULL)
# if Walkable(checkPoint, currentPoint->next)
# // Make a straight path between those points:
# temp = currentPoint
# currentPoint = currentPoint->next
# delete temp from the path
# else
# checkPoint = currentPoint
# currentPoint = currentPoint->next

def Walkable(node1, node2):
    pass
def smoothPath(path):
    path = list(path)
    checkPoint = path[0]
    i = 1
    currentPoint = path[i]
    
    while path[i+1] != None:
        if(i+2) >= len(path):
            break
        print(i)
        if Walkable(checkPoint, path[i+1]):
            path.remove(path[i])
        else:
            checkPoint = currentPoint
            currentPoint = path[i+1]
            i = i + 1
    return path

path = smoothPath(path)

NameError: name 'path' is not defined

In [ ]:
#code to display the new map
w = 649*2
h = 319*2

#initialize pygame
m2024 = Map(image_name, "Crescendo", arr)

pygame.init()
screen = pygame.display.set_mode((w,h))
imp = pygame.image.load(m2024.get_image()).convert()

#Main booleans
isPressed = False
draw = True
run = True
screen.blit(imp, (0, 0))

#Show_img, shows the obstacles
def show_img(screen, arr):
    for i in range(59):
        for c in range(29):
            if arr[i][c] == 1:
                x = int(i*22)
                y = int(c*22)
                pygame.draw.rect(screen, (0, 0, 0),
                 [x, y, 22, 22], 0)
            # if arr[i][c] == 2:
            #     x = int(i*22)
            #     y = int(c*22)
            #     pygame.draw.rect(screen, (255, 0, 0),
            #      [x, y, 22, 22], 0)
            

startbool = True
endbool = False
showex = showey = showsx = showsy = 0
path = []
while run:
    screen.fill((0,0,0))
    screen.blit(imp, (0, 0))
    #Keybinds
    keys=pygame.key.get_pressed()
    #Define start
    if keys[pygame.K_s] == True:
        startbool = True
        endbool = False
    #Define end
    elif keys[pygame.K_e] == True:
        startbool = False
        endbool = True
    #A* intialization
    elif keys[pygame.K_i] == True:
        print(start, end)
        #Open and Closed list
        openlist = []
        closedlist = []
    
        #Start Node and End Node
        start_node = Node(None, start)
        start_node.g = start_node.h = start_node.f = 0
        end_node = Node(None, end)
        end_node.g = end_node.h = end_node.f = 0
    
        #Add to the heapqueue
        heappush(openlist, start_node)
        i = 0
        while len(openlist) > 0:
            #Draw open and closed lists
            screen.fill((0,0,0))
            screen.blit(imp, (0, 0))
            show_img(screen, arr)
            pygame.draw.rect(screen, (0, 255, 255),
                    [showsx*22, showsy*22, 22, 22], 0)
            pygame.draw.rect(screen, (0, 255, 255),
                        [showex*22, showey*22, 22, 22], 0)
            
            breakbool, openlist, closedlist, final_node = astarloop(arr, openlist,closedlist, end)
            
            for node in openlist:
                pygame.draw.rect(screen, (0, 255, 0),
                        [node.position[0]*22, node.position[1]*22, 22, 22], 0)
            for node in closedlist:
                #print("draw rect closed: " + str(node.f))
                #print("it be drawn")
                pygame.draw.rect(screen, (255, 0, 0),
                        [node.position[0]*22, node.position[1]*22, 22, 22], 0)
                
            #If you find the path, return it and draw it
            if breakbool == True:
                print('path found')
                path = []
                path.insert(0,final_node.position)
                node = final_node.parent
                while node.position != start:
                    path.insert(0, node.position)
                    node = node.parent
                for node in path:
                    print(str(node))
                    pygame.draw.rect(screen, (0 ,255, 255),
                        [node[0]*22, node[1]*22, 22, 22], 0)
                pygame.display.update()
                pygame.time.wait(2000)
                break
            pygame.display.update()
            
    #Draw the path
    elif keys[pygame.K_p] == True:
        for i in range(len(path)-1):   
            node1pos = (path[i][0]*22)+10, (path[i][1]*22+10)
            node2pos = ((path[i+1][0]*22)+10, (path[i+1][1]*22)+10)
            pygame.draw.line(screen, (0,255,255), node1pos, node2pos, 2)
    elif keys[pygame.K_q] == True:
        path = smoothPath(path)
            
        
    show_img(screen, m2024.get_arr())
    
    #Show start and end nodes
    if showex != 0 and showey != 0:
        pygame.draw.rect(screen, (0, 255, 255),
                        [showex*22, showey*22, 22, 22], 0)
    if showsx != 0 and showsy != 0:
        pygame.draw.rect(screen, (0, 255, 255),
                    [showsx*22, showsy*22, 22, 22], 0)
            
        
    for event in  pygame.event.get():
        if event.type == pygame.QUIT:
            run = False
        if event.type == pygame.MOUSEBUTTONDOWN:
            if startbool == True:
                (showsx, showsy) = pygame.mouse.get_pos()
                showsx = int(showsx/22)
                showsy = int(showsy/22)
                sx = int(showsx)
                sy = int(showsy)
                print("start set", sx, sy)
                start = (sx,sy)
            if endbool == True:
                (showex, showey) = pygame.mouse.get_pos()
                showex = int(showex/22)
                showey = int(showey/22)
                ex = showex
                ey = showey
                print("end set", ex, ey)
                end = (ex,ey)
            
    pygame.display.update()
            
pygame.quit()

start set 5 17
end set 30 23
(5, 17) (30, 23)
path found
(6, 17)
(7, 17)
(8, 17)
(9, 17)
(10, 17)
(11, 17)
(12, 17)
(13, 17)
(14, 17)
(15, 18)
(16, 19)
(17, 20)
(18, 21)
(19, 21)
(20, 21)
(21, 21)
(22, 21)
(23, 21)
(24, 21)
(25, 21)
(26, 21)
(27, 22)
(28, 22)
(29, 22)
(30, 23)
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


: 

In [10]:
#find gateways (typically middle) unless middle doesn't work
#find paths between gateways
        
def findGateways(arr):
    r = 0
    c = 0
    for c in range(29):
        for r in range(58):
            if arr[10+(11*r),6+(11*c)] != 0 or arr[10+(11*r)+1,6+(11*c)] != 0:
                nc = 6+(11*c)
                while True:
                    nc = nc - 1
                    if arr[10+(11*r), nc] == 0 and arr[10+(11*r)+1, nc] == 0:
                        arr[10+(11*r),nc] = 2
                        arr[10+(11*r)+1,nc] = 2
                        break
            else:
                arr[10+(11*r),6+(11*c)] = 2
                arr[10+(11*r)+1,6+(11*c)] = 2
                
    return arr
arr = findGateways(arr)


In [31]:
arr1 = np.empty((int(649/11),int(319/11)))
arr1[0,0] = arr[0:11,0:11]

ValueError: setting an array element with a sequence.

In [ ]:
arr1 = np.zeroes((int(649/11),int(319/11)))

#find box that start and end are in
#run a* on the larger boxes
#arr1.state()


def heirarchicalastar(arr, start, end):
    #find which box start and end are in
    #a* that finds paths based on that
    openlist = []
    closedlist = []
    
    start_node = Node(None, start)
    start_node.g = start_node.h = start_node.f = 0
    end_node = Node(None, end)
    end_node.g = end_node.h = end_node.f = 0
    
    heappush(openlist, start_node)
    
    while len(openlist) > 0:
        breakbool, openlist, closedlist, final_node = astarloop(arr, openlist, closedlist, end)
    
        if breakbool == True:
            path = []
            path.insert(0,final_node.position)
            node = final_node.parent
            while node != start:
                path.insert(0, node.position)
                node = node.parent
            return path